In [3]:
########################################
# meta knn-* using all of raw features #
########################################
import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import math

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

drop_cols = ['id', 'visit_date', 'visitors', 'hpg_store_id', 'fold', 'air_store_id', 
             'air_store_id_encoded', 'hpg_store_id_encoded', 
             'air_reserved_visitors', 'hpg_reserved_visitors','reserved_visitors']

DataBaseDir = '../../data'
InputDir = '%s/l0/kfold' % DataBaseDir
OutputDir = '%s/meta/kfold' % DataBaseDir
kfold = 5
neighor_range = 6
use_selected = False
neighbor_num_list = [int(math.pow(2, i)) for i in list(range(7, 11))]
#### load data
valid_dfs = []
holdout_dfs = []
test_dfs = []
for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    valid = pd.read_csv('%s/valid.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    holdout = pd.read_csv('%s/holdout.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    test = pd.read_csv('%s/test.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    valid['fold'] = fold
    valid_dfs.append(valid)
    holdout_dfs.append(holdout)
    test_dfs.append(test)
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True) 
##### model selection with CV
start = time.time()
for nn in neighbor_num_list:
    strategy = 'knn_%s' % nn
    # score
    cv_score = .0
    holdout_score = .0
    for fold in range(kfold):
        FoldData = {
            'train': TrainData[TrainData['fold'] != fold],
            'valid': TrainData[TrainData['fold'] == fold],
            'holdout': holdout_dfs[fold],
            'test': test_dfs[fold]
        }
        if(use_selected):
            selected_features = []
            with open('../../data/gfs/en_good_features.txt', 'r') as i_file:
                for line in i_file:
                    selected_features.append(line.rstrip())
            i_file.close()
#             selected_features.extend(meta_feats)
            #selected_features.extend(['air_store_id_encoded', 'hpg_store_id_encoded'])
            col = selected_features
        else:
            col = [c for c in FoldData['train'].columns if c not in drop_cols]
#             col = [c for c in col if((c.startswith('inter') == False))]
        print('feature size %s' % len(col))
        model = neighbors.KNeighborsRegressor(n_jobs= 2, n_neighbors= nn)
        model.fit(FoldData['train'][col].values, FoldData['train']['visitors'].values)
        # for valid
        FoldData['valid'][strategy] = model.predict(FoldData['valid'][col].values)
        rmsle_valid = RMSLE(FoldData['valid']['visitors'].values, FoldData['valid'][strategy])
        cv_score += rmsle_valid
        # for holdout
        FoldData['holdout'][strategy] = model.predict(FoldData['holdout'][col])
        rmsle_holdout = RMSLE(FoldData['holdout']['visitors'].values, FoldData['holdout'][strategy])
        holdout_score += rmsle_holdout
        # for test
        FoldData['test'][strategy] = model.predict(FoldData['test'][col])
        #### output
        FoldOutputDir = '%s/%s' % (OutputDir, fold)
        if(os.path.exists(FoldOutputDir) == False):
            os.makedirs(FoldOutputDir)
        for mod in FoldData.keys():
            if(mod == 'train'):
                continue
            OutCols = []
            if(mod == 'test'):
                OutCols.append('id')
            OutCols.extend(['air_store_id', 'visit_date', 'visitors', strategy])
            OutputFile = '%s/%s_%s.csv' % (FoldOutputDir, mod, strategy)
            OutFoldData = FoldData[mod][OutCols]
            OutFoldData.to_csv(OutputFile, index= False)
        print('fold %s: valid score %.6f, holdout score %.6f, valid length %s' % (fold, rmsle_valid, rmsle_holdout, len(FoldData['valid'])))  
    cv_score /= kfold # Average valid set predictions
    holdout_score /= kfold # Average holdout set predictions
    end = time.time()
    print('\n======================')
    print("meta model KNN-%s done, CV score %.4f, Holdout score %.4f, Elapsed time: %.2fs" % (nn, cv_score, holdout_score, (end - start)))
    print('======================\n')

/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


feature size 254


/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


fold 0: valid score 0.708147, holdout score 0.705422, valid length 45371
feature size 254
fold 1: valid score 0.705310, holdout score 0.706241, valid length 45371
feature size 254
fold 2: valid score 0.705898, holdout score 0.705494, valid length 45371
feature size 254
fold 3: valid score 0.711132, holdout score 0.704832, valid length 45370
feature size 254
fold 4: valid score 0.707985, holdout score 0.706349, valid length 45370

meta model KNN-128 done, CV score 0.7077, Holdout score 0.7057, Elapsed time: 648.66s

feature size 254
fold 0: valid score 0.725629, holdout score 0.722479, valid length 45371
feature size 254
fold 1: valid score 0.722182, holdout score 0.722793, valid length 45371
feature size 254
fold 2: valid score 0.722800, holdout score 0.722176, valid length 45371
feature size 254
fold 3: valid score 0.728827, holdout score 0.721998, valid length 45370
feature size 254
fold 4: valid score 0.724696, holdout score 0.722862, valid length 45370

meta model KNN-256 done, CV 